In [ ]:
import os

os.environ['TRANSFORMERS_OFFLINE'] = '1'

In [ ]:
import torch
from transformers import AutoTokenizer
from models import DecoderOnlyTransformer
from config import *
from generate_utils import *
from utils import *

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(CONFIG['tokenizer'])
CONFIG['vocab_size'] = tokenizer.vocab_size

config_check()
model = DecoderOnlyTransformer(CONFIG).to(CONFIG['device'])
model.apply_attention_map()
model.load_state_dict(torch.load('ckpts/DecoderOnlyTransformer.pth', weights_only=True), strict=False)

In [ ]:
GENERATE_CONFIG['temperature'] = 0.5
GENERATE_CONFIG['greedy'] = False

context = "The government"
context_generate(context, model, tokenizer, max_new_tokens=25)

In [ ]:
plot_attention(model, layer=11, batch_idx=0)